In [ ]:
#Library
import pandas as pd
import numpy as np
import datetime as dt
import string
import time
import matplotlib.pyplot as plt
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import PiecewiseAggregateApproximation
from tslearn.piecewise import SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
from math import sqrt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from langdetect import detect
import matplotlib.pyplot as plt
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [ ]:
#Tweet Sentiment Analysis

tweetsdf = pd.read_csv('q1tweetstockpot.csv')

tweetsdf['text'] = tweetsdf['text'].astype(str)
tweetsdf['lang'] = tweetsdf['text'].map(lambda x: detect(x)) #takes awhile to process
tweetsdf = tweetsdf[tweetsdf['lang']=='en']

analyzer = SentimentIntensityAnalyzer()
sentiment = tweetsdf['text'].apply(lambda x: analyzer.polarity_scores(x)) 
df4 = pd.concat([tweetsdf,sentiment.apply(pd.Series)],1)
df4.info()
df4.describe()

# Create sentiment polarity classes
df4['summary'] = df4['compound'].apply(lambda x: (x>0 and 'Positive') or (x<0 and 'Negative') or 'Neutral')
df4.to_csv('rawsentitweets.csv',index=False)

df5 = df4.groupby(['name','timestamp'])['compound'].mean().reset_index(name = 'avg_senti')
df5['tweetsenticlass']=df5['avg_senti'].apply(lambda x: (x>0 and 'Positive') or (x<0 and 'Negative') or 'Neutral')
df5.to_csv('finalsentitweets.csv',index=False)



In [ ]:
# Merge to create Final Dataset 

dfori = pd.read_csv('potentialstock.csv')
dfnewsenti = pd.read_csv('finalsentinews.csv')
dfnewsenti.rename(columns={'dates':'day'}, inplace=True)
dftweetsenti = pd.read_csv('finalsentitweets.csv')
dftweetsenti.rename(columns={'timestamp':'day'}, inplace=True)

merged_df_union = pd.merge(dfori,dfnewsenti,on=['name','day'],how='left')
merged_df_union['avg_senti'].fillna(0, inplace=True)
merged_df_union['newssenticlass'].fillna('none', inplace=True)

merged_df_union2 = pd.merge(merged_df_union,dftweetsenti,on=['name','day'],how='left')
merged_df_union2['avg_senti_y'].fillna(0, inplace=True)
merged_df_union2['tweetsenticlass'].fillna('none', inplace=True)

merged_df_union2.to_csv('mergestocksq1.csv',index=False)
